In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from train_utils import run_epoch, eval_rw
from blender_dataloader import BlenderDataloader
from rw_dataloader import RealWorldDataloader

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


In [3]:
iterates_to_combine = list(range(48, 56)) #[7, 8, 9, 10]
model_stem = '12.15'
#run_id = 'kszjvkvy'

In [4]:
#glob.glob(f"{BESPOKE_ROOT}/models/m_{model_stem}*")

In [5]:
m_paths = [f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch" for e in iterates_to_combine]
m_paths

['/home/beans/bespoke/models/m_12.15_e48.torch',
 '/home/beans/bespoke/models/m_12.15_e49.torch',
 '/home/beans/bespoke/models/m_12.15_e50.torch',
 '/home/beans/bespoke/models/m_12.15_e51.torch',
 '/home/beans/bespoke/models/m_12.15_e52.torch',
 '/home/beans/bespoke/models/m_12.15_e53.torch',
 '/home/beans/bespoke/models/m_12.15_e54.torch',
 '/home/beans/bespoke/models/m_12.15_e55.torch']

In [6]:
state_dicts = [torch.load(p) for p in m_paths]
state_dict_avg = state_dicts[0].copy() # placeholder for a sec

In [7]:
for k in state_dict_avg.keys():
    state_dict_avg[k] = sum([sd[k] for sd in state_dicts]) / len(state_dicts)

In [8]:
m = EffNet().to(device)
m.load_state_dict(state_dict_avg)

<All keys matched successfully>

In [9]:
for mm in m.modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .005
        print(mm.momentum)

0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005
0.005


In [10]:
BS = 64
dataloader = BlenderDataloader(path_stem="trn", bs=BS)

In [11]:
%%time
with torch.no_grad():
    run_epoch(dataloader, m, train=True, backwards=False, log_wandb=False, updates_per_epoch=256)

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


AttributeError: 'NoneType' object has no attribute 'param_groups'

In [12]:
for mm in m.modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .1 # set this back to value used for train. Actually is this even necessary? is this stored in state_dict?
        #print(mm.momentum)

In [13]:
torch.save(m.state_dict(), f'{BESPOKE_ROOT}/models/m_{model_stem}_avg.torch')

In [15]:
rw_dataloader = RealWorldDataloader()

(5400, 100, 640, 3) (5400, 5) (5400, 1)
(6000, 100, 640, 3) (6000, 5) (6000, 1)


In [16]:
import wandb
wandb.init(id=run_id, project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [17]:
eval_rw(rw_dataloader, m, wandb)

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
